# RAG Pipeline


In [1]:
!pip install numpy==1.26.4 # downgrading the numpy version

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 86.6 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2


After installing 1.26.4, you need to "Restart Session" and re-import numpy. It's strange that they haven't updated their release notes yet (https://colab.research.google.com/notebooks/relnotes.ipynb)

In [1]:
import numpy
import pandas as pd

In [4]:
!pip install faiss-cpu
!pip install faiss-gpu-cu12 # CUDA 12.x, Python 3.8+
!pip install dotenv
!pip install langchain_chroma
!pip install langchain-community
!pip install langchain_experimental
!pip install langchain_openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 72.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.9/47.9 MB 41.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 6.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.1/611.1 kB 38.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 85.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 19.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 78.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 83.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.9/55.9 kB 5.4 MB/s eta 0:00

In [5]:
import os
import torch
import pandas as pd
from tqdm import tqdm
from typing import List
import logging

# Configure logging
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")

def load_text_files(path: str) -> List[str]:
    """
    Load text files from the given path.

    Args:
        path (str): The path to the directory or file containing the text files.

    Returns:
        list: A list of text documents.
    """
    docs = []

    try:
        if os.path.isdir(path):
            # Iterate over files in the directory
            for file_name in os.listdir(path):
                if file_name.endswith(".txt"):
                    file_path = os.path.join(path, file_name)
                    with open(file_path, 'r', encoding='utf-8') as file:
                        docs.append(file.read())
        elif os.path.isfile(path) and path.endswith(".txt"):
            # If the path is a file, directly read it
            with open(path, 'r', encoding='utf-8') as file:
                docs.append(file.read())
    except Exception as e:
        logging.error(f"Error loading text files from {path}: {e}")

    return docs

def format_retrieved_docs(docs: List[str]) -> str:
    """
    Format the retrieved documents in reverse order.

    Args:
        docs (list): A list of documents.

    Returns:
        str: Formatted string with contexts.
    """
    try:
        docs = reversed(docs)
        return "\n\n".join([f"Context {i+1}: {doc}" for i, doc in enumerate(docs)])
    except Exception as e:
        logging.error(f"Error formatting retrieved documents: {e}")
        return ""

def rerank_docs(query: str, retriever, rerank_model_name: str, k: int = 3) -> List[str]:
    """
    Rerank the retrieved documents based on the query using Flashrank.

    Args:
        query (str): The query to rerank documents for.
        retriever: The base retriever object.
        rerank_model_name (str): The name of the rerank model.
        k (int): The number of top documents to rerank.

    Returns:
        list: A list of reranked documents.
    """
    try:
        compressor = FlashrankRerank(top_n=k, model=rerank_model_name)
        compression_retriever = ContextualCompressionRetriever(
            base_compressor=compressor, base_retriever=retriever
        )
        return compression_retriever.invoke(query)
    except Exception as e:
        logging.error(f"Error reranking documents for query '{query}': {e}")
        return []

def get_hypo_doc(query: str, generation_pipe) -> str:
    """
    Generate a hypothesis document for the given query using the language model.

    Args:
        query (str): The query to generate a hypothesis for.
        generation_pipe: The language model pipeline.

    Returns:
        str: The hypothesis document or the original query if unavailable.
    """
    template = """Imagine you are an expert providing a detailed and factual explanation in response to the query '{query}'.
    Your response should include all key points that would be found in a top search result, without adding any personal opinions, commentary, or experiences.
    Do not include any subjective phrases such as 'I think', 'I believe', or 'I am not sure'. Do not apologize, hedge, or express uncertainty.
    The response should be structured as an objective, factual explanation only, without any conversational elements or chatting.
    If you are truly uncertain and cannot provide an accurate answer, simply respond with: 'Unavailable: {query}'.
    Otherwise, answer confidently with only the relevant information.
    """

    messages = [{"role": "user", "content": template.format(query=query)}]

    try:
        with torch.no_grad():
            hypo_doc = generation_pipe(messages, max_new_tokens=100, return_full_text=False)[0]["generated_text"]

        logging.info(f"Generated hypothesis document for query: {query}")
        if hypo_doc.startswith("Unavailable"):
            logging.warning(f"Hypothesis unavailable for query: {query}")
            return query
        return hypo_doc
    except Exception as e:
        logging.error(f"Error generating hypothesis document for query '{query}': {e}")
        return query

def answer_generation(
    qa_df: pd.DataFrame, output_file: str, retriever, generation_pipe,
    prompt, rerank: bool, rerank_model_name: str, hypo: bool, top_k_rerank: int = 3
):
    """
    Generate answers for the given questions using the retriever and the generation pipeline.

    Args:
        qa_df (pd.DataFrame): DataFrame containing questions and other metadata.
        output_file (str): Path to save the generated answers.
        retriever: A retriever object to retrieve documents.
        generation_pipe: A pipeline object for text generation.
        prompt: A ChatPromptTemplate object for generating prompts.
        rerank (bool): Whether to rerank retrieved documents.
        rerank_model_name (str): The name of the rerank model.
        hypo (bool): Whether to generate a hypothesis document.
        top_k_rerank (int): Number of top documents to rerank.
    """
    logging.info("Starting answer generation...")

    # Check if the output file exists
    if not os.path.exists(output_file):
        with open(output_file, 'w') as f_out:
            f_out.write(",".join(list(qa_df.columns) + ["Generated_Answer"]) + "\n")
            start_idx = 0
    else:
        # Calculate the number of rows in the output file
        with open(output_file, 'r') as f_out:
            num_rows = sum(1 for line in f_out)
            start_idx = num_rows - 1

    # Iterate over the DataFrame
    with open(output_file, 'a') as f_out:
        for idx, row in tqdm(qa_df.iterrows(), total=len(qa_df)):
            if idx < start_idx:
                continue

            query = row["Question"]
            if hypo:
                query = get_hypo_doc(query, generation_pipe)

            # Retrieve documents
            try:
                if rerank:
                    logging.info(f"Reranking documents for query: {query}")
                    retrieved_docs = rerank_docs(query, retriever, rerank_model_name, k=top_k_rerank)
                else:
                    retrieved_docs = retriever.invoke(query)
            except Exception as e:
                logging.error(f"Error retrieving documents for query '{query}': {e}")
                continue

            # Format the documents
            context = format_retrieved_docs(retrieved_docs)

            # Create the full prompt
            prompt_messages = prompt.format_messages(context=context, question=row["Question"])
            full_prompt = "\n".join(message.content for message in prompt_messages)

            # Generate the answer
            try:
                messages = [{"role": "user", "content": full_prompt}]
                with torch.no_grad():
                    llm_output = generation_pipe(
                        messages, max_new_tokens=20, return_full_text=False
                    )[0]["generated_text"]

                row["Generated_Answer"] = llm_output
                pd.DataFrame([row]).to_csv(f_out, header=False, index=False)
            except Exception as e:
                logging.error(f"Error generating answer for query '{query}': {e}")
                continue

            # Clear cache
            del retrieved_docs, context, prompt_messages, full_prompt, messages, llm_output
            torch.cuda.empty_cache()


# Constants
PROMPT_TEMPLATE = """
You are an expert assistant answering factual questions about Pittsburgh or Carnegie Mellon University (CMU).
Use the retrieved information to give a detailed and helpful answer. If the provided context does not contain the answer, leverage your pretraining knowledge to provide the correct answer.
If you truly do not know, just say "I don't know."

Important Instructions:
- Answer concisely without repeating the question.
- Use the provided context if relevant; otherwise, rely on your pretraining knowledge.
- Do **not** use complete sentences. Provide only the word, name, date, or phrase that directly answers the question. For example, given the question "When was Carnegie Mellon University founded?", you should only answer "1900".

Examples:
Question: Who is Pittsburgh named after?
Answer: William Pitt
Question: What famous machine learning venue had its first conference in Pittsburgh in 1980?
Answer: ICML
Question: What musical artist is performing at PPG Arena on October 13?
Answer: Billie Eilish

Context: \n\n {context} \n\n
Question: {question} \n\n
Answer:
"""

In [ ]:
import os
import torch
import pandas as pd
from tqdm import tqdm
import faiss
import numpy as np
import pickle
import random
from dotenv import load_dotenv
from huggingface_hub import login
from transformers import pipeline, AutoModelForCausalLM, AutoTokenizer
from langchain_chroma import Chroma
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import (
    RecursiveCharacterTextSplitter,
    CharacterTextSplitter,
    TokenTextSplitter
)
from langchain_experimental.text_splitter import SemanticChunker
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.docstore.document import Document
from langchain.prompts import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    PromptTemplate
)

# ========================================
# Helper Functions
# ========================================
def str2bool(value):
    if isinstance(value, bool):
        return value
    if value.lower() in ('yes', 'true', 't', 'y', '1'):
        return True
    elif value.lower() in ('no', 'false', 'f', 'n', '0'):
        return False
    else:
        raise ValueError('Boolean value expected.')

# ========================================
# Main Function for Jupyter/IPython
# ========================================
def run_RAG(
    model_name = "meta-llama/Llama-3.1-8B-Instruct",
    dtype = "float16", # or torch.bfloat16
    embedding_model_name = "sentence-transformers/all-MiniLM-L6-v2",
    embedding_dim = 384,
    splitter_type = "recursive", # or "character", "token", "semantic"
    chunk_size = 1000,
    chunk_overlap = 200,
    text_files_path = "./data/scraped/scraped_all",
    sublink_files_path = "./data/scraped/scraped_text_data",
    sublink_files_nums = 0,
    retriever_type = "FAISS", # or "CHROMA"
    retriever_algorithm = "similarity", # or "mmr"
    rerank = False,
    rerank_model_name = "ms-marco-MultiBERT-L-12",
    top_k_search = 3,
    top_k_rerank = 3,
    hypo = False,
    qes_file_path = "./data/annotated/QA_pairs_1.csv",
    output_file = "./output/results.json",
    qa_nums = 100
):
    # Step 0: Load environment variables
    load_dotenv()

    os.environ["LANGCHAIN_TRACING_V2"] = "true"
    os.environ["LANGCHAIN_API_KEY"] = os.getenv('LANGCHAIN_API_KEY') # os.getenv('LANGCHAIN_API_KEY')
    os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
    os.environ["LANGCHAIN_PROJECT"] = "rag-template"
    os.environ["USER_AGENT"] = "LangChain/1.0 (+https://www.langchain.com)"

    login(token=os.getenv('HUGGINGFACE_TOKEN')) # os.getenv('HUGGINGFACE_TOKEN')

    # Set model name, precision, and other parameters
    dtype = torch.float16 if dtype == "float16" else torch.bfloat16
    random.seed(42)

    # Check if rerank is set to True
    if rerank:
        print("Reranking is set to True.")

    # Step 1: Initialize the Hugging Face model as your LLM
    print("Initializing the Hugging Face model...")
    model = AutoModelForCausalLM.from_pretrained(
        model_name, torch_dtype=dtype, device_map="cuda:0"
    )

    tokenizer = AutoTokenizer.from_pretrained(model_name)
    tokenizer.pad_token = tokenizer.eos_token
    tokenizer.padding_side = "left"

    generation_pipe = pipeline(
        "text-generation",
        model=model,
        tokenizer=tokenizer,
        torch_dtype=dtype
    )
    print("Model initialized successfully!")

    # Step 2: Load the Sentence Transformers model for embeddings
    docs_length = f"main160_sublink{sublink_files_nums}"
    model_name_str = embedding_model_name.split('/')[-1]
    embeddings_file_path = f"./data/embeddings/embeddings_{model_name_str}_{docs_length}_{splitter_type}_{chunk_size}_{chunk_overlap}.npy"
    splits_file_path = f"./data/embeddings/splits_{model_name_str}_{docs_length}_{splitter_type}_{chunk_size}_{chunk_overlap}.pkl"
    embeddings = None
    splits = None
    embedding_model = HuggingFaceEmbeddings(model_name=embedding_model_name)
    print(f"Start loading QA from {qes_file_path}")
    qa_test_data_path = qes_file_path
    qa_df = pd.read_csv(qa_test_data_path)
    print(len(qa_df))
    if len(qa_df) != 574:
        qa_df = qa_df.sample(qa_nums, random_state=221)
    print(f"Loaded {len(qa_df)} QAs")

    if not os.path.exists(embeddings_file_path):
        # Step 3: Load the text files for building the index and QA evaluation
        print(f"Start loading texts from {text_files_path}")
        docs = load_text_files(path=text_files_path)
        documents = [Document(page_content=text) for text in tqdm(docs, desc="Wrapping text in Document objects")]
        del docs

        if sublink_files_nums != 0:
            sublink_file_store_path = "./data/embeddings/sublink_docs.pkl"
            if os.path.exists(sublink_file_store_path):
                print(f"Start loading sublink files from {sublink_file_store_path}")
                with open(sublink_file_store_path, "rb") as f:
                    all_sublink_docs = pickle.load(f)
            else:
                print(f"Start reading all sublink files")
                all_sublink_docs = load_text_files(path=sublink_files_path)
                print(f"Finish loading {len(all_sublink_docs)} sublinks, now store it")
                with open(sublink_file_store_path, 'wb') as f:
                    pickle.dump(all_sublink_docs, f)
                print(f"Store all sublink file in {sublink_file_store_path}")

            sampled_sublink_docs = random.sample(all_sublink_docs, sublink_files_nums)
            documents.extend([Document(page_content=text) for text in tqdm(sampled_sublink_docs, desc="Wrapping text in Document objects")])
            del sampled_sublink_docs
            del all_sublink_docs

        if splitter_type == "recursive":
            text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
        elif splitter_type == "character":
            text_splitter = CharacterTextSplitter(separator=" ", chunk_size=chunk_size, chunk_overlap=chunk_overlap)
        elif splitter_type == "token":
            text_splitter = TokenTextSplitter(chunk_size=int(chunk_size / 4), chunk_overlap=int(chunk_overlap / 4))
        elif splitter_type == "semantic":
            text_splitter = SemanticChunker(
                embeddings=embedding_model,
                breakpoint_threshold_type="percentile",
                breakpoint_threshold_amount=80
            )
        else:
            raise ValueError("Invalid splitter type. Please choose between recursive, character, token, or semantic.")

        splits = text_splitter.split_documents(documents)
        del documents
        print(f"End splitting texts -- Number of splits: {len(splits)}")

        embeddings = embedding_model.embed_documents([doc.page_content for doc in tqdm(splits, desc="Embedding texts")])
        print(f"End embedding texts")
        torch.cuda.empty_cache()
        print(f"Start saving embeddings and splits")
        np.save(embeddings_file_path, embeddings)
        with open(splits_file_path, 'wb') as f:
            pickle.dump(splits, f)
        print(f"Embeddings saved in {embeddings_file_path}, splits saved in {splits_file_path}")
    else:
        print("Embeddings already exist! Loading embeddings")
        embeddings = np.load(embeddings_file_path)
        with open(splits_file_path, 'rb') as f:
            splits = pickle.load(f)
        print("End loading")

    # Step 6: Create the RAG prompting pipeline
    prompt_template = PromptTemplate(
        input_variables=['context', 'question'],
        template=PROMPT_TEMPLATE
    )
    human_message_template = HumanMessagePromptTemplate(prompt=prompt_template)
    chat_prompt_template = ChatPromptTemplate(
        input_variables=['context', 'question'],
        messages=[human_message_template]
    )
    prompt = chat_prompt_template

    # Step 7: Generate answers for the questions
    print("Building the vectorstore...")
    if retriever_type == "CHROMA":
        vectorstore = Chroma.from_documents(documents=splits, embedding=embedding_model, collection_name="collectionChroma")
        retriever = vectorstore.as_retriever(search_type=retriever_algorithm, search_kwargs={'k': top_k_search})
    elif retriever_type == "FAISS":
        retriever = FAISS.from_documents(splits, embedding_model).as_retriever(search_type=retriever_algorithm, search_kwargs={"k": top_k_search})
    else:
        raise ValueError("Invalid retriever type. Please choose between FAISS or CHROMA.")

    print("Retriever built successfully!")
    torch.cuda.empty_cache()
    del splits

    answer_generation(
        qa_df, output_file, retriever,
        generation_pipe, prompt, rerank, rerank_model_name, hypo, top_k_rerank=top_k_rerank
    )

    print(f"QA evaluation completed! Results saved to {output_file}")

run_RAG(
    model_name="meta-llama/Llama-3.1-8B-Instruct",
    dtype="float16",
    embedding_model_name="sentence-transformers/all-MiniLM-L6-v2",
    text_files_path="./data/scraped/scraped_text_data",
    qes_file_path="./data/annotated/QA_pairs_1.csv",
    output_file="./output/results.json",
    qa_nums=100
)

Initializing the Hugging Face model...


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/855 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

...

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Start loading QA from ./data/annotated/QA_pairs_1.csv
3938
Loaded 100 QAs
Embeddings already exist! Loading embeddings
End loading
Building the vectorstore...
Retriever built successfully!


100%|██████████| 100/100 [01:03<00:00,  1.58it/s]

QA evaluation completed! Results saved to ./output/results.json
